In [1]:
import pandas as pd
from tqdm import tqdm
import time

import numpy as np
np.random.seed(3)

import joblib
import plotly.express as px

import warnings
warnings.simplefilter('ignore')

import re
tqdm.pandas()

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer

import spacy

from gensim import models
from tqdm import tqdm
import numpy as np

import plotly.graph_objects as go

from sklearn.feature_extraction.text import CountVectorizer

from gensim import corpora
from p_tqdm import p_map

In [2]:
df=pd.read_excel('/home/ubuntu/MyNotebooks/Data/wipo_ai_processed2.xlsx')

In [3]:
df.head()

,Application Id,Application Year,Full Text,Applicant,type
0,US42597370,2000,bacterial strains genes and enzymes for contro...,Institute Of Molecular Agrobiology,Research Institute
1,US39458207,2000,reversible rolling method and reversible rolli...,Hitachi,Company
2,WO2001024205,2000,composite magnetic sheet and method of produci...,Tokin Corporation,Company
3,US39488862,2000,compounding assembly for nutritional fluids an...,Baxter International Inc,Company
4,US39579616,2000,hydrotreating catalyst for hydrotreating hydro...,Tonen Corporation,Company


In [4]:
nlp = spacy.load('en_core_web_md') #initializing stopwords
stop_words = list(spacy.lang.en.stop_words.STOP_WORDS) +['present','disclosure','method', 'system', 'use', 'useful', 'overall', 'have', 'having', 'said', 'relates', 'embodiment','methods','invention','consists','disclosed','inventions','apparatus','include','includes','provide','consist','comprise','comprises','includes','including','provided']

In [5]:
def process_text(text, stem=False): #clean text
    
    text = re.sub('[^A-Za-z]', ' ', text.lower()) #remove non-alphabets
    
    clean_text = [
         str(word.lemma_) for word in nlp(text)
         if word.pos_ in ['PROPN','NOUN','ADJ'] and str(word) not in stop_words
    ]
    clean_text=[x for x in clean_text if len(x)>1]
    return ' '.join(clean_text)

In [6]:
clean_text=p_map(process_text, df['Full Text'].astype(str).tolist())

In [7]:
from gensim.models.phrases import Phrases, Phraser

text_tokens=[x.split() for x in clean_text]

phrases=Phrases(text_tokens, min_count=5)

In [8]:
tokens=phrases[text_tokens]
for token in tokens[100:105]:
    print(token)

['single_chain', 'antibody', 'mutant', 'mutation', 'protein', 'conformational_change', 'exposure', 'epitope', 'hydrophobic', 'core', 'molecule', 'single_chain', 'antibody_scfv', 'common', 'mutant', 'epitope', 'mutant', 'wild_type', 'dna_molecule', 'scfv', 'pharmaceutical_composition', 'antibody', 'treatment', 'pharmaceutical_composition']
['vacuum', 'processing', 'equipment', 'wafer', 'vacuum', 'processing', 'equipment', 'multiple', 'set', 'succession', 'different', 'process', 'individual', 'wafer_wafer', 'processing', 'set', 'transport', 'set', 'wafer', 'processing', 'route', 'multiple', 'set', 'processing', 'possible', 'set', 'processing', 'process', 'valid_invalid', 'status', 'operation', 'processing', 'operational_status', 'invalid', 'processing', 'route', 'processing', 'operational_status', 'valid', 'wafer', 'processing', 'valid', 'operation']
['image', 'processing', 'example', 'order', 'conversion', 'smooth', 'skin', 'wrinkle', 'enhancement', 'natural', 'look', 'grain', 'texture'

In [9]:
df['clean_text']=clean_text
df['phrases']=tokens
df.to_excel('wipo_processed_phrases.xlsx', index=False)